# Aggregating & Reshaping Data Frames

grouping columns, performing aggregation calculations, and pivoting & unpivoting data

Topics:
 - Grouping Columns
 - Aggregating Groups
 - Melting DataFrames
 - Multi-Index DataFrames
 - Pivot Tables

Goals:

 - Group DataFrames by one or more columns and calculate aggregate statistics by group
 - Learn to access multi-index DataFrames and reset them to return to a single index
 - Create Excel-style PivotTables to summarize data
 - Melt "wide" tables of data into a "long" tabular form
 

In [70]:
# import libraries
import pandas as pd

In [72]:
premier_league = pd.read_csv('../data/premier-league/premier-league-matches.csv')

In [73]:
premier_league.head()

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A


In [74]:
# data analysis pattern

# DataFrame Shape
print("DataFrame Shape:", premier_league.shape)

DataFrame Shape: (12026, 8)


In [75]:

# Display first few rows
# Qual a dimensionalidade do dado?
print("Head of the DataFrame:")
premier_league.head(8)

Head of the DataFrame:


,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
5,1993,1,1992-08-15,Ipswich Town,1,1,Aston Villa,D
6,1993,1,1992-08-15,Everton,1,1,Sheffield Weds,D
7,1993,1,1992-08-15,Southampton,0,0,Tottenham,D


In [76]:
# Display last few rows
print("Tail of the DataFrame:")
premier_league.tail()

Tail of the DataFrame:


,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
12021,2023,38,2023-05-28,Everton,1,0,Bournemouth,H
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham,H
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton,H
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham,A
12025,2023,38,2023-05-28,Brentford,1,0,Manchester City,H


In [77]:
# Data Types of each column
# Every column is actually a series and each series has a type
print("Data Types:")
premier_league.dtypes

Data Types:


Season_End_Year     int64
Wk                  int64
Date               object
Home               object
HomeGoals           int64
AwayGoals           int64
Away               object
FTR                object
dtype: object

In [78]:
# Descriptive Statistics
print("Descriptive Statistics:")
premier_league.describe()

Descriptive Statistics:


,Season_End_Year,Wk,HomeGoals,AwayGoals
count,12026.000000,12026.000000,12026.000000,12026.000000
mean,2007.713620,19.730501,1.524364,1.142525
std,9.072559,11.123916,1.306417,1.133930
min,1993.000000,1.000000,0.000000,0.000000
25%,2000.000000,10.000000,1.000000,0.000000
50%,2008.000000,20.000000,1.000000,1.000000
75%,2016.000000,29.000000,2.000000,2.000000
max,2023.000000,42.000000,9.000000,9.000000


In [79]:
# Aggregation functions

premier_league.sum()

Season_End_Year                                             24144764
Wk                                                            237279
Date               1992-08-151992-08-151992-08-151992-08-151992-0...
Home               Coventry CityLeeds UnitedSheffield UtdCrystal ...
HomeGoals                                                      18332
AwayGoals                                                      13740
Away               MiddlesbroughWimbledonManchester UtdBlackburnN...
FTR                HHHDADDDDHDHAAHDDHHHAHDAHDDDDHHHAADDDHHHHADAAH...
dtype: object

Uh... is not a good idea at the moment.

So, we should try separate the data by categorical and numerical columns:

In [80]:
# Identifying categorical (object) and numerical columns
categorical_cols = premier_league.select_dtypes(include=['object']).columns
numerical_cols = premier_league.select_dtypes(include=['number']).columns

In [81]:
num_premier_league = premier_league[numerical_cols].copy()

In [82]:
num_premier_league.sum()

Season_End_Year    24144764
Wk                   237279
HomeGoals             18332
AwayGoals             13740
dtype: int64

In [83]:
num_premier_league.mean()

Season_End_Year    2007.713620
Wk                   19.730501
HomeGoals             1.524364
AwayGoals             1.142525
dtype: float64

Ok, this is better but, the id column didn't make sense in that point of view. So, we can better describe the numerical columns(instead id) using the `.groupby()` method.

## The .groupby() method

The `groupby()` function in pandas is used to split the data into groups based on some criteria. In essence, it helps perform "split-apply-combine" operations:

1. **Split**: Split the data into groups based on some criteria (e.g., the values in one or more columns).
2. **Apply**: Apply a function to each group independently.
3. **Combine**: Combine the results into a DataFrame or Series.

### Standad format:

`data_frame.groupby(column_group_list)[column_operation_list].aggreg_operation()`

 - ``data_frame``: The DataFrame you are working with.
 - ``column_group_list``: A single column name or a list of column names by which you want to group the data. This defines the grouping criteria.
 - ``column_operation_list``: A single column name or a list of column names on which you want to apply the aggregation operation.
 - ``aggreg_operation()``: The aggregation function you want to apply, such as ``mean(), sum(), count(), min(), max()``, etc.

### Code Explanation

1. **Group by the away team and calculate the average away goals**:

    ```python
    away_goals_mean = premier_league.groupby("Away")[['AwayGoals']].mean()
    ```

    - `premier_league.groupby("Away")`: This part of the code groups the `premier_league` DataFrame by the "Away" column, which contains the names of the away teams.
    - `[["AwayGoals"]]`: This selects the "AwayGoals" column from the DataFrame for further operations.
    - `.mean()`: This calculates the mean (average) of the "AwayGoals" for each group of away teams.

    After this line, `away_goals_mean` is a new DataFrame where each row corresponds to an away team, and the single column contains the average number of goals scored by that team in away games.

2. **Sort by the average away goals in descending order**:

    ```python
    away_goals_mean_sorted = away_goals_mean.sort_values('AwayGoals', ascending=False)
    ```

    - `away_goals_mean.sort_values('AwayGoals', ascending=False)`: This sorts the `away_goals_mean` DataFrame by the "AwayGoals" column in descending order. The `ascending=False` argument specifies that the sorting should be in descending order.

    After this line, `away_goals_mean_sorted` is a new DataFrame where the rows are sorted such that the teams with the highest average away goals appear first.

In [84]:
# Group by the away team and calculate the average away goals
away_goals_mean = premier_league.groupby("Away")[['AwayGoals']].mean()

In [88]:
away_goals_mean.head(10)

,AwayGoals
Away,
Arsenal,1.509244
Aston Villa,1.113383
Barnsley,0.631579
Birmingham City,0.834586
Blackburn,1.097701
Blackpool,1.315789
Bolton,1.036437
Bournemouth,1.070175
Bradford City,0.578947


In [86]:
# Sort by the average away goals in descending order
away_goals_mean_sorted = away_goals_mean.sort_values('AwayGoals', ascending=False)

In [87]:
#Three best visitors ?
away_goals_mean_sorted.head(3)

,AwayGoals
Away,
Manchester Utd,1.668908
Arsenal,1.509244
Liverpool,1.502521


In [89]:
def group_aggregating(data_frame, column_group_list, column_operation_list, aggreg_operation, ascending=False):
    """
    Groups the data_frame by columns in column_group_list and performs aggregation operations 
    on columns in column_operation_list using aggreg_operation, then sorts the result.

    Parameters:
    data_frame (pd.DataFrame): The DataFrame to perform operations on.
    column_group_list (list or str): The column(s) to group by.
    column_operation_list (list or str): The column(s) to apply aggregation operations on.
    aggreg_operation (str): The aggregation operation to apply (e.g., 'mean', 'sum').
    ascending (bool): Whether to sort the result in ascending order (default is False).

    Returns:
    pd.DataFrame: The resulting DataFrame after grouping, aggregating, and sorting.
    """
    
    # Group by specified columns
    grouped = data_frame.groupby(column_group_list)[column_operation_list]
    
    # Apply the aggregation operation
    aggregated = getattr(grouped, aggreg_operation)()
    
    # Sort the results
    sorted_result = aggregated.sort_values(by=column_operation_list, ascending=ascending)
    
    return sorted_result

In [109]:
# Example usage
result_by_year = group_aggregating(premier_league, ['Season_End_Year','Home'], ['HomeGoals', 'AwayGoals'], 'mean', ascending=[True,True])

In [110]:
result_by_year = result_by_year.sort_values(by='Season_End_Year', ascending=True)
result_by_year

HomeGoals  AwayGoals
Season_End_Year Home                                
1993            Manchester Utd   1.857143   0.666667
                Wimbledon        1.523810   1.095238
                Crystal Palace   1.285714   1.190476
                Aston Villa      1.714286   0.761905
                Blackburn        1.809524   0.857143
...                                   ...        ...
2023            Brentford        1.842105   0.947368
                Southampton      1.000000   1.947368
                Liverpool        2.421053   0.894737
                Crystal Palace   1.105263   1.210526
                Bournemouth      1.052632   1.473684

[626 rows x 2 columns]

In [55]:
# Example usage
result = group_aggregating(premier_league, ['Home'], ['HomeGoals', 'AwayGoals'], 'mean', ascending=False)
result

                 HomeGoals  AwayGoals
Home                                 
Manchester Utd    2.100840   0.736134
Manchester City   2.034000   0.950000
Arsenal           2.028571   0.863866
Liverpool         2.020168   0.798319
Chelsea           1.917647   0.847059
Tottenham         1.734454   1.080672
Newcastle Utd     1.617537   1.128731
Oldham Athletic   1.595238   1.500000
Blackpool         1.578947   1.947368
Blackburn         1.566092   1.094828
Everton           1.507563   1.121008
Brentford         1.500000   1.026316
Sheffield Weds    1.481013   1.183544
Leeds United      1.477663   1.123711
West Ham          1.462282   1.274662
Leicester City    1.409231   1.280000
Southampton       1.400433   1.253247
QPR               1.395683   1.366906
Portsmouth        1.383459   1.195489
Wimbledon         1.379747   1.253165
Middlesbrough     1.369338   1.236934
Bournemouth       1.368421   1.543860
Fulham            1.351974   1.289474
Swansea City      1.345865   1.278195
Aston Villa 

In [56]:
premier_league.columns

Index(['Season_End_Year', 'Wk', 'Date', 'Home', 'HomeGoals', 'AwayGoals',
       'Away', 'FTR'],
      dtype='object')

In [57]:
# Create HomeWin column
premier_league['HomeWin'] = premier_league.apply(lambda row: 1 if (row['HomeGoals'] - row['AwayGoals']) > 0 else 0, axis=1)

# Create AwayWin column
premier_league['AwayWin'] = premier_league.apply(lambda row: 1 if (row['AwayGoals'] - row['HomeGoals']) > 0 else 0, axis=1)

In [58]:
premier_league.head()

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,HomeWin,AwayWin
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H,1,0
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H,1,0
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H,1,0
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D,0,0
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A,0,1


In [59]:
# Find the top 3 teams with the most home wins for each season
top_home_wins = premier_league.groupby(['Season_End_Year', 'Home'])['HomeWin'].sum().reset_index()
top_home_wins = top_home_wins.sort_values(['Season_End_Year', 'HomeWin'], ascending=[True, False])

In [65]:
top_home_wins = top_home_wins.groupby('Season_End_Year').head(1)

In [61]:
# Find the top 3 teams with the most away wins for each season
top_away_wins = premier_league.groupby(['Season_End_Year', 'Away'])['AwayWin'].sum().reset_index()
top_away_wins = top_away_wins.sort_values(['Season_End_Year', 'AwayWin'], ascending=[True, False])

In [66]:
top_away_wins = top_away_wins.groupby('Season_End_Year').head(1)

In [67]:
print("Top 3 teams with the most home wins for each season:")
top_home_wins

Top 3 teams with the most home wins for each season:


,Season_End_Year,Home,HomeWin
11,1993,Manchester Utd,14
24,1994,Blackburn,14
46,1995,Blackburn,17
78,1996,Newcastle Utd,17
98,1997,Newcastle Utd,13
106,1998,Arsenal,15
126,1999,Arsenal,14
156,2000,Manchester Utd,15
166,2001,Arsenal,15
186,2002,Arsenal,12


In [68]:
print("\nTop team with the most away wins for each season:")
top_away_wins


Top team with the most away wins for each season:


,Season_End_Year,Away,AwayWin
11,1993,Manchester Utd,10
32,1994,Manchester Utd,13
46,1995,Blackburn,10
76,1996,Manchester Utd,10
86,1997,Arsenal,9
119,1998,Manchester Utd,10
126,1999,Arsenal,8
156,2000,Manchester Utd,13
174,2001,Ipswich Town,9
186,2002,Arsenal,14


In [69]:
premier_league

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,HomeWin,AwayWin
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H,1,0
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H,1,0
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H,1,0
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D,0,0
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A,0,1
...,...,...,...,...,...,...,...,...,...,...
12021,2023,38,2023-05-28,Everton,1,0,Bournemouth,H,1,0
12022,2023,38,2023-05-28,Leicester City,2,1,West Ham,H,1,0
12023,2023,38,2023-05-28,Aston Villa,2,1,Brighton,H,1,0
12024,2023,38,2023-05-28,Leeds United,1,4,Tottenham,A,0,1
